# 抽出型の要約の実装

In [1]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumeval.metrics.bleu import BLEUCalculator
from sumeval.metrics.rouge import RougeCalculator

In [2]:
%load_ext autoreload
%autoreload 2
pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)

## パラメータの設定

In [3]:
data_dir_path = Path('data')

## 要約データの取得

In [4]:
body_data = pd.read_csv(data_dir_path.joinpath('body_data.csv'))
summary_data = pd.read_csv(data_dir_path.joinpath('summary_data.csv'))

In [5]:
body_data.head()

,article_id,title,text
0,11036614,NaN,NaN
1,11489315,太陽170億個分もの超大質量ブラックホールがありえない場所で発見される,\nこれまで発見された超大質量のブラックホールは、いずれも「銀河の密集地帯」から発見されてい...
2,11560663,NaN,NaN
3,11481499,「一緒に帰ろう」はNG！気になる彼と仲良くなるための4つの鉄則,\n気になる男子と仲良くなろうと思って、勇気を振り絞って「今日、一緒に帰ろうよ」と言ったとこ...
4,11545109,「40代以降で結婚できる人は1.2％」厳しすぎるアラフォーの婚活事情,\n「40代以降で結婚できる男性は1.2％、女性は2.7％」(※)。この数字を見て、心が折れ...


In [6]:
body_data = body_data.query('text.notnull()', engine='python').reset_index(drop=True)

In [7]:
summary_data = pd.merge(
    summary_data,
    body_data[['article_id']],
    on='article_id', how='inner'
)

In [8]:
body_data.shape

(13462, 3)

## ニュースの要約

In [9]:
index = 1
target_data = body_data.iloc[index]
article_id = target_data['article_id']
body_text = target_data['text']
summary_original_texts = summary_data.query(f'article_id == {article_id}', engine='python')['text'].tolist()

print('\n'.join(summary_original_texts))
print()
print(body_text)

「気になる彼と仲良くなるための鉄則」を、恋愛上手な女子に聞きました！
「一緒に帰ろう」より「一緒に行こう」と誘い、共通の目的を持つといいそう
過去の話をして、共通点を見出すのもおすすめとのことです


気になる男子と仲良くなろうと思って、勇気を振り絞って「今日、一緒に帰ろうよ」と言ったところで「ごめん、今日はちょっと用事があって」という返事が返ってきたら哀しいですよね。今回は、何人かの恋愛上手な女子に「気になる彼と仲良くなるための4つの鉄則」についてお話をお聞きしてきました。さっそくご紹介しましょう！■１．一緒に「帰る」ではなく「行く」「一緒に帰ろうと言っても、ちょっと仲良くなりづらいと思います。『帰ろう』というのは、なんとなく言いやすい言葉かもしれませんが、『一緒に行こう』がベターでは？　一緒にごはんに行こうとか、一緒に図書館に行こうとか」（28歳／モデル）
つまり彼と共通の目的を持って行動しましょうということですよね。夜のお店の「同伴」とおなじです。気になる男子と同伴すれば、ごはんのあとに「一緒にお店に行く」という「共通の目的」が生まれます。帰ろう、つまり「アフターしよう」だと、アフターのあとどうしていいのかわからないので、お互いにオロオロ、ドキドキするだけです。■２．一緒にごはんを食べる「気になる彼と仲良くなろうと思えば、ランチでもいいので、絶対に何回も一緒にごはんを食べるべきです」（28歳／役員秘書）ドキドキしながら、なにを食べているのかわからないようなごはんであっても、絶対に何回も一緒にごはんを食べるべきです。ごはんって、ふたりの仲を理屈抜きにかなり親密にしてくれますよ。この「理屈抜きに」というのが、一緒にごはんを食べるという行為の素晴らしさです。■３．親族・家族の話をする「気になる彼と仲良くしたいと思えば、家族とか親戚の話をするといいです」（27歳／看護師）こちらも「理屈抜きに」仲良くなれる行為です。お互いに会っているときの相手しか知らないわけですが、家族や親戚の話というのは、いわばその人のルーツなわけです。共有することで相手の事を深く知ったような気になれますし、距離も縮まる、というわけです。■４．過去の話をする「気になる彼と仲良くなりたければ、高校時代のこととか、前職のこととか、とにかくじぶんの過去をお話するといいです」（28歳／受付）過去の話を聞きつ

In [10]:
LANGUAGE = 'japanese'
SENTENCES_COUNT = 3

In [11]:
parser = PlaintextParser.from_string(body_text, Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
summarizer = LexRankSummarizer(stemmer)

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

■３．親族・家族の話をする「気になる彼と仲良くしたいと思えば、家族とか親戚の話をするといいです」（27歳／看護師）こちらも「理屈抜きに」仲良くなれる行為です。
■４．過去の話をする「気になる彼と仲良くなりたければ、高校時代のこととか、前職のこととか、とにかくじぶんの過去をお話するといいです」（28歳／受付）過去の話を聞きつつ、人は「じぶんとの共通点」を探しているものです。
仲良くなるというのは、じぶんと他者との共通点を見出すことができたということですから、どんどん過去の話をしてみてはいかがでしょうか。


In [12]:
parser = PlaintextParser.from_string(body_text, Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
summarizer = TextRankSummarizer(stemmer)

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

気になる男子と仲良くなろうと思って、勇気を振り絞って「今日、一緒に帰ろうよ」と言ったところで「ごめん、今日はちょっと用事があって」という返事が返ってきたら哀しいですよね。
■２．一緒にごはんを食べる「気になる彼と仲良くなろうと思えば、ランチでもいいので、絶対に何回も一緒にごはんを食べるべきです」（28歳／役員秘書）ドキドキしながら、なにを食べているのかわからないようなごはんであっても、絶対に何回も一緒にごはんを食べるべきです。
■４．過去の話をする「気になる彼と仲良くなりたければ、高校時代のこととか、前職のこととか、とにかくじぶんの過去をお話するといいです」（28歳／受付）過去の話を聞きつつ、人は「じぶんとの共通点」を探しているものです。


In [13]:
parser = PlaintextParser.from_string(body_text, Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
summarizer = LsaSummarizer(stemmer)

for sentence in summarizer(parser.document, SENTENCES_COUNT):
    print(sentence)

共有することで相手の事を深く知ったような気になれますし、距離も縮まる、というわけです。
4つを並べてみると、どれも簡単なテクニックですよね。
いつもおなじことを言うようで恐縮ですが、こういうテクニックって、恋愛上手な女子はもうほとんど本能的にやっていたりします。


## 要約手法の評価

In [14]:
def summarize_text(summarizer, text, LANGUAGE, SENTENCES_COUNT):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    summary_result_texts = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        summary_result_texts.append(sentence.__str__())
    return summary_result_texts

In [15]:
rouge = RougeCalculator(stopwords=True, lang="ja")
bleu = BLEUCalculator(lang="ja")
stemmer = Stemmer(LANGUAGE)

In [16]:
summarize_dict = {
    'lsa': LsaSummarizer,
    'text_rank': TextRankSummarizer,
    'lex_rank': LexRankSummarizer
}

In [17]:
summary_results = []
for i in tqdm(range(len(body_data))):

    target_data = body_data.iloc[i]
    article_id = target_data['article_id']
    body_text = target_data['text']
    summary_original_texts = summary_data.query(f'article_id == {article_id}', engine='python')['text'].tolist()

    for summarize_name, Summarizer in summarize_dict.items():

        summarizer = Summarizer(stemmer)

        summary_result_texts = summarize_text(
            summarizer=summarizer, text=body_text, LANGUAGE=LANGUAGE, SENTENCES_COUNT=SENTENCES_COUNT
        )

        summary_result = ''.join(summary_result_texts)
        summary_original = ''.join(summary_original_texts)

        rouge_1 = rouge.rouge_n(
            summary=summary_result,
            references=summary_original,
            n=1
        )

        rouge_l = rouge.rouge_l(
            summary=summary_result,
            references=summary_original
        )

        bleu_score = bleu.bleu(summary_result, summary_original)

        summary_results.append([article_id, summarize_name, rouge_1, rouge_l, bleu_score])

summary_results = pd.DataFrame(
    summary_results,
    columns=['article_id', 'summarize_name', 'rouge_1', 'rouge_l', 'bleu_score']
)

100%|██████████| 13462/13462 [21:14<00:00, 10.56it/s] 


In [18]:
plot_data = summary_results.copy()
plot_data['rank_rouge1'] = plot_data.groupby('article_id')['rouge_1'].rank(ascending=False)
plot_data['rank_rougel'] = plot_data.groupby('article_id')['rouge_l'].rank(ascending=False)
plot_data['rank_bleu'] = plot_data.groupby('article_id')['bleu_score'].rank(ascending=False)

plot_data = plot_data[['article_id', 'summarize_name', 'rank_rouge1', 'rank_rougel', 'rank_bleu']].set_index(
    ['article_id', 'summarize_name']
).stack().reset_index()
plot_data.columns = ['article_id', 'summarize_name', 'algorithm_name', 'rank']
plot_data = plot_data.groupby(['algorithm_name', 'summarize_name']).agg({
    'rank': ['mean', 'median']
})
plot_data.columns = list(map(lambda x: x[1], plot_data.columns))
plot_data = plot_data.reset_index().pivot_table(
    index='algorithm_name',
    columns='summarize_name'
)
plot_data

mean                       median               
summarize_name  lex_rank       lsa text_rank lex_rank  lsa text_rank
algorithm_name                                                      
rank_bleu       1.819158  2.048655  2.132187      2.0  2.0       2.0
rank_rouge1     1.770316  2.011105  2.218578      2.0  2.0       2.0
rank_rougel     1.823800  2.067449  2.108751      2.0  2.0       2.0

In [19]:
plot_data = summary_results.copy()
plot_data['rank_rouge1'] = plot_data.groupby('article_id')['rouge_1'].rank(ascending=False)
plot_data['rank_rougel'] = plot_data.groupby('article_id')['rouge_l'].rank(ascending=False)
plot_data['rank_bleu'] = plot_data.groupby('article_id')['bleu_score'].rank(ascending=False)

plot_data = plot_data[['article_id', 'summarize_name', 'rank_rouge1', 'rank_rougel', 'rank_bleu']].set_index(
    ['article_id', 'summarize_name']
).stack().reset_index()
plot_data.columns = ['article_id', 'summarize_name', 'algorithm_name', 'rank']
plot_data = plot_data.groupby(['algorithm_name', 'summarize_name', 'rank'])['article_id'].count().reset_index().rename(
    columns={'article_id': 'n_data'}
)
plot_data['total_n_data'] = plot_data.groupby(['algorithm_name', 'summarize_name'])['n_data'].transform('sum')
plot_data = plot_data.assign(data_rate=lambda x: x.n_data / x.total_n_data).pivot_table(
    index=['algorithm_name', 'summarize_name'],
    columns='rank'
)
plot_data

data_rate                                \
rank                                1.0       1.5       2.0       2.5   
algorithm_name summarize_name                                           
rank_bleu      lex_rank        0.347274  0.028228  0.432402  0.023102   
               lsa             0.269202  0.018125  0.375279  0.020948   
               text_rank       0.175680  0.025553  0.462561  0.031125   
rank_rouge1    lex_rank        0.370747  0.026445  0.435448  0.026148   
               lsa             0.272842  0.019759  0.403060  0.021022   
               text_rank       0.148641  0.025553  0.429580  0.032462   
rank_rougel    lex_rank        0.342148  0.027559  0.437825  0.025479   
               lsa             0.255534  0.020725  0.379958  0.020874   
               text_rank       0.192096  0.028228  0.449636  0.030159   

                                        n_data                        \
rank                                3.0    1.0  1.5   2.0  2.5   3.0   
algorithm_name summarize_name                                          
rank_bleu      lex_rank        0.168994   4675  380  5821  311  2275   
               lsa             0.316446   3624  244  5052  282  4260   
               text_rank       0.305081   2365  344  6227  419  4107   
rank_rouge1    lex_rank        0.141212   4991  356  5862  352  1901   
               lsa             0.283316   3673  266  5426  283  3814   
               text_rank       0.363765   2001  344  5783  437  4897   
rank_rougel    lex_rank        0.166989   4606  371  5894  343  2248   
               lsa             0.322909   3440  279  5115  281  4347   
               text_rank       0.299881   2586  380  6053  406  4037   

                              total_n_data                              
rank                                   1.0    1.5    2.0    2.5    3.0  
algorithm_name summarize_name                                           
rank_bleu      lex_rank              13462  13462  13462  13462  13462  
               lsa                   13462  13462  13462  13462  13462  
               text_rank             13462  13462  13462  13462  13462  
rank_rouge1    lex_rank              13462  13462  13462  13462  13462  
               lsa                   13462  13462  13462  13462  13462  
               text_rank             13462  13462  13462  13462  13462  
rank_rougel    lex_rank              13462  13462  13462  13462  13462  
               lsa                   13462  13462  13462  13462  13462  
               text_rank             13462  13462  13462  13462  13462

## Extracting Keywords

In [22]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [24]:
text = '''
The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.
'''

tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(10)

science - 1.7521671497168656
fiction - 1.7168390247168657
China - 1.4722488091138661
Earth - 1.4143046707606364
Wandering - 1.1038853114478115
tone - 1.0971675275482093
fans - 1.0971675275482093
weekend - 1.0310056818181819
America - 1.0260545033670034
North - 1.0076557239057238
throwback - 1.0019926346801344
budget - 1.0014987038950105
